## Sujet N°1. 🚚 Optimisation de Chaîne Logistique Multi-Entrepôts

#### Étape 1 : Modélisation des Données (Structures dict)

In [41]:
entrepots = {
    'Entrepot_A': {
        'stock': {'ProduitX': 50, 'ProduitY': 30, 'ProduitZ': 20},
        'cout_transport': 10
    },
    'Entrepot_B': {
        'stock': {'ProduitX': 40, 'ProduitY': 60, 'ProduitZ': 10},
        'cout_transport': 8
    },
    'Entrepot_C': {
        'stock': {'ProduitX': 30, 'ProduitY': 20, 'ProduitZ': 50},
        'cout_transport': 12
    }
}

commande_client = {'ProduitX': 90, 'ProduitY': 80, 'ProduitZ': 70}

#### Étape 2 : Coûts Variables (Introduction à Sympy)

In [42]:
import sympy

fonction_cout_str = "2 * quantite + 5"  

quantite = sympy.symbols('quantite')
expression_cout = sympy.sympify(fonction_cout_str)
fonction_cout = sympy.lambdify(quantite, expression_cout)

print(f"Fonction de coût utilisée: {fonction_cout_str}")

# Exemple 

print(fonction_cout(10)) ## ==> 2 * 10 + 5 = 25

Fonction de coût utilisée: 2 * quantite + 5
25


#### Étape 3 : Algorithme d'Optimisation "Glouton" (Fonctions, for, if)

In [43]:
def trouver_solution(commande, entrepots):
    plan_approvisionnement = {}
    cout_total = 0

    for produit, quantite_voulue in commande.items():
        
        entrepots_disponibles = []
        
        for nom, infos in entrepots.items():
            if produit in infos['stock'] and infos['stock'][produit] > 0:
                entrepots_disponibles.append((nom, infos))

        # Trier du moins cher au plus cher
        entrepots_disponibles.sort(key=lambda e: e[1]['cout_transport'])

        quantite_restante = quantite_voulue

        for nom, infos in entrepots_disponibles:
            if quantite_restante <= 0:
                break

            stock_dispo = infos['stock'][produit]
            quantite_prendre = min(stock_dispo, quantite_restante)

            infos['stock'][produit] -= quantite_prendre

            if produit not in plan_approvisionnement:
                plan_approvisionnement[produit] = []
            plan_approvisionnement[produit].append((nom, quantite_prendre))

            print(f"  → Prendre {quantite_prendre} chez {nom}")

            quantite_restante -= quantite_prendre

        if quantite_restante > 0:
            print(f" Il manque {quantite_restante} unités de {produit}")

    return plan_approvisionnement, cout_total


#### Étape 4 : Calcul Final et Affichage (Intégration, print)

In [44]:
def trouver_solution(commande, entrepots, fonction_cout):
    plan_approvisionnement = {}
    cout_total = 0

    for produit, quantite_voulue in commande.items():
        print(f"\nProduit : {produit} (Quantité demandée : {quantite_voulue})")

        entrepots_disponibles = []
        for nom, infos in entrepots.items():
            if produit in infos['stock'] and infos['stock'][produit] > 0:
                entrepots_disponibles.append((nom, infos))

        entrepots_disponibles.sort(key=lambda e: e[1]['cout_transport'])

        quantite_restante = quantite_voulue

        for nom, infos in entrepots_disponibles:
            if quantite_restante <= 0:
                break  
            
            # Quantité disponible dans cet entrepôt
            stock_dispo = infos['stock'][produit]

            # On prend le minimum entre ce qu'on veut et ce qu'il y a en stock
            quantite_prendre = min(stock_dispo, quantite_restante)

            # Mettre à jour le stock de l'entrepôt
            infos['stock'][produit] -= quantite_prendre

            # Ajouter cette action au plan
            if produit not in plan_approvisionnement:
                plan_approvisionnement[produit] = []
            plan_approvisionnement[produit].append((nom, quantite_prendre))

            # Calcul du coût :
            cout_variable = fonction_cout(quantite_prendre)  # coût variable selon quantité
            cout_transport = infos['cout_transport']          # coût fixe du transport
            cout_total += cout_variable + cout_transport      # ajout au coût total

            # Afficher le détail
            print(f"  → Prendre {quantite_prendre} unités chez {nom} | "
                  f"Coût variable = {cout_variable:.2f}, transport = {cout_transport:.2f}")

            # Réduire la quantité restante
            quantite_restante -= quantite_prendre

        # Si la commande n’a pas pu être complètement remplie
        if quantite_restante > 0:
            print(f" Il manque {quantite_restante} unités de {produit}")

    # Afficher le résumé final
    print("=== Résumé du plan d'approvisionnement ===")
    for produit, details in plan_approvisionnement.items():
        for entrepot, qte in details:
            print(f"- {produit} : {qte} depuis {entrepot}")
    print(f"Coût total final : {cout_total:.2f} €")

    # Retourner le plan et le coût
    return plan_approvisionnement, cout_total


In [45]:
# Appel de la fonction
plan, cout_total = trouver_solution(commande_client, entrepots, fonction_cout)



Produit : ProduitX (Quantité demandée : 90)
  → Prendre 40 unités chez Entrepot_B | Coût variable = 85.00, transport = 8.00
  → Prendre 50 unités chez Entrepot_A | Coût variable = 105.00, transport = 10.00

Produit : ProduitY (Quantité demandée : 80)
  → Prendre 60 unités chez Entrepot_B | Coût variable = 125.00, transport = 8.00
  → Prendre 20 unités chez Entrepot_A | Coût variable = 45.00, transport = 10.00

Produit : ProduitZ (Quantité demandée : 70)
  → Prendre 10 unités chez Entrepot_B | Coût variable = 25.00, transport = 8.00
  → Prendre 20 unités chez Entrepot_A | Coût variable = 45.00, transport = 10.00
  → Prendre 40 unités chez Entrepot_C | Coût variable = 85.00, transport = 12.00
=== Résumé du plan d'approvisionnement ===
- ProduitX : 40 depuis Entrepot_B
- ProduitX : 50 depuis Entrepot_A
- ProduitY : 60 depuis Entrepot_B
- ProduitY : 20 depuis Entrepot_A
- ProduitZ : 10 depuis Entrepot_B
- ProduitZ : 20 depuis Entrepot_A
- ProduitZ : 40 depuis Entrepot_C
Coût total final :